# Import Libraires

In [8]:
# %pip install openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\simon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# import some basic python libaries (operating system, regular expressions, time)
import time
from datetime import datetime
import json
import logging
# import some selenium components to rememeneber by name
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException,NoAlertPresentException,StaleElementReferenceException

#import mongoDB, we may add additional databases in the future for better 

from pymongo import MongoClient

from bs4 import BeautifulSoup

#import custom library 


# Setup

In [3]:

#Setup url
#url = "https://www.glassdoor.ca/Job/canada-internship-software-engineer-jobs-SRCH_IL.0,6_IN3_KO7,35.htm"
url = "https://www.glassdoor.ca/Job/remote-mobile-developer-jobs-SRCH_IL.0,6_IS12228_KO7,23.htm"# url for wisdom
# Create a MongoClient to the running mongod instance
client = MongoClient("mongodb://localhost:27017")
db = client["my_database"]
#Delete all documents in the collection before we start 
collection = db['opportunities_glassdoor']
result = collection.delete_many({})
print(result.deleted_count, " documents deleted.")



0  documents deleted.


# Functions

In [4]:
def handle_possible_popup(driver):
    """
    Function that safely handles possible popups that may appear on this page.

    Parameters:
        driver (WebDriver): The WebDriver object
    """    
    try:
        # Wait for the modal to appear and find the close button
        close_button = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@role='dialog']//button[@class='CloseButton']"))
        )
        # Click the close button
        close_button.click()
    except (TimeoutException, NoSuchElementException):
        # No alert button found or unable to click it, safely pass
        pass
    except Exception as e:
        # Log the unexpected exception for debugging
        print(f"An unexpected error occurred: {e}")
        pass

# Data Ingestion

In [5]:
# Configure webdriver object and ActionsChain object
driver = webdriver.Chrome()
actions = ActionChains(driver)

# Open the website
driver.get(url)
time.sleep(2)

# First, find the 'Show More Jobs' button and click it until all jobs are open
while True:
    try:
        show_more_jobs_button = driver.find_element(By.XPATH, "//button[@data-test='load-more']")
        if show_more_jobs_button.is_displayed():
            actions.move_to_element(show_more_jobs_button).perform()
            show_more_jobs_button.click()
            time.sleep(1)
            handle_possible_popup(driver)
        else:
            break
    except (NoSuchElementException, IndexError):
        # If the button is not found, break the loop
        print("No more 'Show More Jobs' button found.")
        break
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        break
# All jobs are now open on the page

# Get a clickable link to all the jobs on the page
all_jobs = driver.find_elements(By.CLASS_NAME, "JobsList_jobListItem__wjTHv")
links = driver.find_elements(By.XPATH, "//a[contains(@class, 'JobCard_trackingLink__GrRYn')]")
print(f"Number of jobs found: {len(all_jobs)}")
# Loop through all the jobs and extract the details
for job in all_jobs:
    try:
        # Click each job
        actions.move_to_element(job).perform()
        #print(job.text)
        time.sleep(1)
        job.click()
        time.sleep(1)
        handle_possible_popup(driver)
        time.sleep(1)
        # Extract and proccees job details here
        show_desc_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='JobDetails_showMore___Le6L'] span")))
        show_desc_button.click()
        time.sleep(1)
        description = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "JobDetails_jobDetailsContainer__y9P3L")))
        description_text = description.text

        company_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//h4[@aria-live='polite']")))
        company_name_text = company_name.text

        position_title = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//h1[@aria-live='polite']")))
        position_title_text = position_title.text
        
        location = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@data-test='location']")))
        location_text = location.text

        apply_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div//header//button[@type='button']//span[1]")))
        apply_button_text = apply_button.text
        
        
        current_window = ""
        if ("Apply on employer site" in apply_button_text ):
            # Click the apply button, save url then close page
            ActionChains(driver).key_down(Keys.CONTROL).click(apply_button).key_up(Keys.CONTROL).perform() #can be replaced with a normal click proabably
            time.sleep(5)
            #check for popups
            try:
                skip_for_now_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-test="skip-registration"]')))
                skip_for_now_button.click()
                print("Skip for now button found.")
                time.sleep(5)
            except (NoSuchElementException, IndexError):
                print("No skip for now button found.")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                pass
                # if windows is opened save url and close
            if (len(driver.window_handles) > 1):
                # Get the current window handle
                driver.switch_to.window(driver.window_handles[-1])
                current_window = driver.current_url
                # Switch to the new window
                # Close the current window
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        else:
            print ("Easy apply found, no link to apply")
                
            # if (len(driver.window_handles) > 1):
            #     # switch to the new window
            #     driver.switch_to.window(driver.window_handles[-1])
            #     # Get the current window handle
            #     current_window = driver.current_url
            #     # Close the current window
            #     time.sleep(2)
            #     driver.close()
            # #if page didnt open itry again
            # else:  
            #     time.sleep(2)
            #     ActionChains(driver).key_down(Keys.CONTROL).click(apply_button).key_up(Keys.CONTROL).perform()
            #     # switch to the new window
            #     driver.switch_to.window(driver.window_handles[-1])
            #     # Get the current window handle
            #     current_window = driver.current_url
            #     # Close the current window
            #     time.sleep(2)
            #     driver.close()
        

        # print (apply_button_text)
        # print(current_window)
        # print (description_text)
        # Insert the job details into the database
        data = {
            "title": position_title_text,
            "company": company_name_text,
            "location": location_text,
            "apply_link": current_window,
            "description": description_text,
        }
        print(data)
        collection.insert_one(data)
    except Exception as e:
        print(f"Error occurred: {e}")
        
        

# Close driver
time.sleep(2)
driver.quit()

No more 'Show More Jobs' button found.
Number of jobs found: 123
Easy apply found, no link to apply
{'title': 'Mobile App Developer at Design Match', 'company': 'Design Match', 'location': 'Remote', 'apply_link': '', 'description': 'Design Match\nMobile App Developer at Design Match\nRemote\nEasy Apply\nEasy Apply\nAre you a talented Mobile App Developer with a flair for creating exceptional mobile experiences? Design Match is looking for a skilled individual to join our team. As a Mobile App Developer, you will be responsible for designing and developing innovative applications for Android and iOS platforms.\nResponsibilities:\nCollaborate with UX designers to translate wireframes and prototypes into functional mobile applications.\nDevelop mobile apps using native languages (Java/Kotlin for Android, Swift for iOS) or cross-platform frameworks (React Native, Flutter).\nOptimize app performance and usability for a seamless user experience.\nConduct rigorous testing to identify and reso